In [3]:
import numpy as np
import shutil,os
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import classification_report
from pylab import interp
import random

In [14]:
def show_img(img,bins_):
    fig = plt.figure()
    ax1 = fig.add_subplot(121)
    ax1.hist(img,bins = bins_,histtype='bar',rwidth=0.8)
    ax2 = fig.add_subplot(122)
    ax2.imshow(img.reshape(80,-1),cmap='gray')
    plt.show()

In [15]:
def histeq(img,nbr_bins):
    imhist,bins = np.histogram(img*(nbr_bins-1),nbr_bins,normed= True)
    cdf = imhist.cumsum()
    cdf = (nbr_bins-1)*cdf/cdf[-1]
    img2 = interp(img*(nbr_bins-1),bins[:-1],cdf)
    return img2/(nbr_bins-1)

In [2]:
train_img = np.fromfile('data/processed/train_img',dtype = 'float64')
train_img = train_img.reshape(-1,6400)
train_label = np.fromfile('data/processed/train_label',dtype = 'int32')
test_img = np.fromfile('data/processed/test_img',dtype = 'float64')
test_img = test_img.reshape(-1,6400)
testdir = 'data/raw/test'
testlist = os.listdir(testdir)

In [86]:
gamma = 0.75
val_idx = np.array(random.sample(range(4384),200))
train_idx = np.setdiff1d(np.arange(4384),val_idx)

In [92]:
knn = KNN(n_neighbors = 5,weights = 'uniform',n_jobs = -1)
knn.fit(train_img[train_idx]**gamma,train_label[train_idx])
result = knn.predict(train_img[val_idx]**gamma)
print(classification_report(train_label[val_idx],result,digits = 5))

             precision    recall  f1-score   support

          0    0.77966   0.93878   0.85185        49
          1    0.97872   0.91391   0.94521       151

avg / total    0.92995   0.92000   0.92233       200



In [93]:
trainset = histeq(train_img,256)

In [107]:
val_idx = np.array(random.sample(range(4384),200))
train_idx = np.setdiff1d(np.arange(4384),val_idx)

In [117]:
knn = KNN(n_neighbors = 5,weights = 'uniform',n_jobs = -1)
knn.fit(trainset[train_idx],train_label[train_idx])
result = knn.predict(trainset[val_idx])
print(classification_report(train_label[val_idx],result,digits = 5))

             precision    recall  f1-score   support

          0    0.90196   0.88462   0.89320        52
          1    0.95973   0.96622   0.96296       148

avg / total    0.94471   0.94500   0.94483       200



In [118]:
testset = histeq(test_img,256)

In [154]:
knn = KNN(n_neighbors = 5,weights = 'uniform',n_jobs = -1)
knn.fit(train_img**1.25,train_label)
result = knn.predict(test_img**1.25)

In [155]:
for i in range(len(result)):
    if result[i] == 0:
        shutil.copy(testdir+'/'+testlist[i],'data/0')
    else:
        shutil.copy(testdir+'/'+testlist[i],'data/1')

In [156]:
lines = [testlist[i]+'    '+str(result[i])+'\n' for i in range(200)]
with open('results.txt','w') as file:
    file.writelines(lines)